In [ ]:
# --- 診断用スクリプト ---
import pandas as pd
import sqlite3
from tqdm.auto import tqdm

# --- 設定（元のスクリプトからコピー） ---
DB_PATH = "data/processed/s2orc_filtered.db"
EVAL_PAPERS_FILE = "data/datapapers/sampled/evaluation_data_papers_50_v2.csv"

# --- 調査開始 ---
print("="*50)
print("--- Starting Investigation: Why 46 instead of 50? ---")
print("="*50)

# 1. 評価対象の全データペーパーDOIをロード
df_eval_papers = pd.read_csv(EVAL_PAPERS_FILE)
all_data_paper_dois = tuple(df_eval_papers['cited_datapaper_doi'].unique())

print(f"Found {len(all_data_paper_dois)} unique data paper DOIs in the input file.\n")

valid_queries_count = 0
skipped_queries = []

# 2. 各データペーパーDOIをループでチェック
with sqlite3.connect(DB_PATH) as conn:
    for i, data_paper_doi in enumerate(tqdm(all_data_paper_dois, desc="Checking each DOI")):
        print(f"\n--- [{i+1}/{len(all_data_paper_dois)}] Checking DOI: {data_paper_doi} ---")
        
        # --- 原因1のチェック ---
        # このデータペーパーを引用し、かつ人間が「正解」とアノテーションした論文を全て取得
        query_gt = "SELECT citing_doi FROM positive_candidates WHERE cited_datapaper_doi = ? AND human_annotation_status = 1"
        gt_rows = conn.execute(query_gt, (data_paper_doi,)).fetchall()
        ground_truth_dois = {row[0] for row in gt_rows}
        
        print(f"Found {len(ground_truth_dois)} annotated positive papers.")

        # 条件: 正解論文が2件以上ないと、クエリと残りの正解に分けられない
        if len(ground_truth_dois) < 2:
            reason = f"Skipped: Not enough positive papers found. (Found {len(ground_truth_dois)}, required >= 2)"
            print(f"🔴 {reason}")
            skipped_queries.append({"doi": data_paper_doi, "reason": reason})
            continue # このDOIの処理を中断し、次に進む

        # --- 原因2のチェック ---
        # 1つをクエリとして取り出す
        query_doi = ground_truth_dois.pop()
        
        # クエリ論文のアブストラクトが存在するかチェック
        query_text_res = conn.execute("SELECT abstract FROM papers WHERE doi = ?", (query_doi,)).fetchone()
        
        if not (query_text_res and query_text_res[0]):
            reason = f"Skipped: The selected query paper ({query_doi}) has no abstract."
            print(f"🔴 {reason}")
            skipped_queries.append({"doi": data_paper_doi, "reason": reason})
            continue

        # --- 原因3のチェック（念のため） ---
        #残りの正解論文がDBに存在するか
        if not ground_truth_dois:
            reason = f"Skipped: After popping one query, no ground truth papers remain."
            print(f"🔴 {reason}")
            skipped_queries.append({"doi": data_paper_doi, "reason": reason})
            continue

        # ここまで到達すれば有効なクエリ
        print(f"✅ Success: This DOI will generate a valid query.")
        print(f"   - Query DOI: {query_doi}")
        print(f"   - Number of ground truths: {len(ground_truth_dois)}")
        valid_queries_count += 1

print("\n" + "="*50)
print("--- Investigation Complete ---")
print("="*50)

print(f"\nTotal valid queries that can be generated: {valid_queries_count}")
print(f"Total skipped data paper DOIs: {len(skipped_queries)}")

if skipped_queries:
    print("\n--- Details of Skipped DOIs ---")
    for item in skipped_queries:
        print(f"- DOI: {item['doi']}")
        print(f"  Reason: {item['reason']}")
else:
    print("\nNo DOIs were skipped.")

In [1]:
import pandas as pd
import sqlite3
import os

# --- 1. 設定 ---
DB_PATH = "data/processed/s2orc_filtered.db"
# ▼▼▼ 検証対象の「新しい」評価ファイル ▼▼▼
FINAL_EVAL_PAPERS_FILE = "data/datapapers/sampled/evaluation_data_papers_50_v2.csv"

print(f"--- Verifying Final Evaluation Set: {FINAL_EVAL_PAPERS_FILE} ---")

# --- 2. 新しい評価用データ論文（50件）のリストをロード ---
try:
    df_eval_papers = pd.read_csv(FINAL_EVAL_PAPERS_FILE)
    eval_data_paper_dois = tuple(df_eval_papers['cited_datapaper_doi'].unique())
    
    if len(df_eval_papers) != 50:
        print(f"⚠️ Warning: File does not contain 50 papers. Found: {len(df_eval_papers)}")
    else:
        print(f"✅ Loaded {len(df_eval_papers)} data papers from file.")
        
except Exception as e:
    print(f"❌ Error loading CSV: {e}")
    raise

# --- 3. DBを調査 ---
results = []
try:
    with sqlite3.connect(DB_PATH) as conn:
        for data_paper_doi in eval_data_paper_dois:
            # 評価スクリプト(22b)のロジックと同様に、
            # 正解DOI (Human=1) の数をカウント
            query_gt = "SELECT COUNT(citing_doi) FROM positive_candidates WHERE cited_datapaper_doi = ? AND human_annotation_status = 1"
            count = conn.execute(query_gt, (data_paper_doi,)).fetchone()[0]
            results.append({
                'data_paper_doi': data_paper_doi,
                'human_used_count': count
            })

    # --- 4. 結果の表示 ---
    df_results = pd.DataFrame(results)
    
    # 正解が2件未満のもの（＝評価スクリプトで除外されてしまうもの）を表示
    df_invalid = df_results[df_results['human_used_count'] < 2]
    
    print("\n" + "="*50)
    if not df_invalid.empty:
        print(f"❌ FAILED: Found {len(df_invalid)} data papers with < 2 'Human: Used' entries:")
        print(df_invalid)
        print("   (These papers will be skipped by the evaluation script)")
    else:
        print(f"✅ SUCCESS: All {len(df_results)} papers in the file have 2 or more 'Human: Used' entries.")
        print("   The evaluation script will now run on all 50 queries.")
    print("="*50)
    
    print("\nFull list of counts:")
    print(df_results.to_markdown(index=False))

except Exception as e:
    print(f"💥 An error occurred: {e}")

--- Verifying Final Evaluation Set: data/datapapers/sampled/evaluation_data_papers_50_v2.csv ---
✅ Loaded 50 data papers from file.

✅ SUCCESS: All 50 papers in the file have 2 or more 'Human: Used' entries.
   The evaluation script will now run on all 50 queries.

Full list of counts:
| data_paper_doi                |   human_used_count |
|:------------------------------|-------------------:|
| 10.1007/S11558-019-09344-2    |                 45 |
| 10.5194/ESSD-13-3907-2021     |                 28 |
| 10.1002/GDJ3.44               |                 21 |
| 10.1016/J.DIB.2017.10.024     |                 13 |
| 10.1038/S41597-021-00997-6    |                 11 |
| 10.1038/S41597-019-0236-X     |                 12 |
| 10.5194/ESSD-13-2895-2021     |                  4 |
| 10.1080/20964471.2019.1625528 |                  7 |
| 10.1038/S41597-020-0386-X     |                 11 |
| 10.1038/S41597-020-0520-9     |                  4 |
| 10.1016/J.DIB.2017.04.013     |                  9 